In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
!cat '/content/drive/MyDrive/MasterThesis/Datasets/Offensive2020/fortest/train/HS/13.txt'

@USER @USER @USER يا لطيف.. يا ساتر ..أحمدوا ربكم إنها مستقعدة🌚كيف لو إنها واقفه👌	OFF	NOT_HS


In [4]:
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "/content/drive/MyDrive/MasterThesis/Datasets/Offensive2020/fortest/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
   
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "/content/drive/MyDrive/MasterThesis/Datasets/Offensive2020/fortest/dev",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    
)
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "/content/drive/MyDrive/MasterThesis/Datasets/Offensive2020/fortest/test", batch_size=batch_size
)

print(
    "Number of batches in raw_train_ds: %d"
    % tf.data.experimental.cardinality(raw_train_ds)
)
print(
    "Number of batches in raw_val_ds: %d" % tf.data.experimental.cardinality(raw_val_ds)
)
print(
    "Number of batches in raw_test_ds: %d"
    % tf.data.experimental.cardinality(raw_test_ds)
)

Found 26 files belonging to 2 classes.
Using 21 files for training.
Found 25 files belonging to 2 classes.
Using 5 files for validation.
Found 24 files belonging to 2 classes.
Number of batches in raw_train_ds: 1
Number of batches in raw_val_ds: 1
Number of batches in raw_test_ds: 1


In [5]:
# It's important to take a look at your raw data to ensure your normalization
# and tokenization will work as expected. We can do that by taking a few
# examples from the training set and looking at them.
# This is one of the places where eager execution shines:
# we can just evaluate these tensors using .numpy()
# instead of needing to evaluate them in a Session/Graph context.
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(7):
        print(text_batch.numpy()[i].decode('utf-8').strip())
        print(label_batch.numpy()[i])
        print('--------------------------------')

@USER اعلامي هلالي وش ترجي منه اي والله الى قال محمد العويس حراق🔥🔥🔥حارق قلبك يا زقان يا المطيًويع من حرق عساه للحرق يا رب جعلني اشوف فيك يوم اسود انت وعائلتك يا ابن الكلب	OFF	NOT_HS
0
--------------------------------
يا التاج ع الراس يا السادة<LF>يا مالك الروح وراعيها❤️	NOT_OFF	NOT_HS
1
--------------------------------
فدوه يا بخت فدوه يا زمن واحد منكم يجيبه	NOT_OFF	NOT_HS
1
--------------------------------
@USER يا بجم يا عجم يا نجم كلنا فالهوا سوا	OFF	NOT_HS
0
--------------------------------
@USER @USER @USER يا لطيف.. يا ساتر ..أحمدوا ربكم إنها مستقعدة🌚كيف لو إنها واقفه👌	OFF	NOT_HS
0
--------------------------------
@USER يا جدعان العلق دا لسه بيقولي هذاكر الاسئلة كفاية ، و راح فتح تويتر .. يا فاشل .. يا فاشل يا فاشل يا فاشل 😂😂😂	OFF	NOT_HS
0
--------------------------------
@USER اخخ يا قلببي يا هالحلقه 😩😭♥️ متعه على بكاء على حماس كل المشاعر داخله ببعض	NOT_OFF	NOT_HS
1
--------------------------------


In [22]:
import re
import string
import sys
import argparse
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


def custom_standardization(input_data):
    # lowercase = tf.strings.lower(input_data)
    # stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    print(input_data.print())
    print("-------------------")
    exit
    # input_string = re.sub(r"([\S])\1+", r"\1", input_data)
    stripped_html = tf.strings.regex_replace(input_data, "[a-zA-Z]|\d+|[٠١٢٣٤٥٦٧٨٩]", " ")
    stripped_html = tf.strings.regex_replace(stripped_html, "[.،,\\-_”“٪ًَ]", " ")
    
    stripped_html = tf.strings.regex_replace(stripped_html, "[إأآا]", "ا")
    stripped_html = tf.strings.regex_replace(stripped_html, "ة", "ه")
    stripped_html = tf.strings.regex_replace(stripped_html, "([\S])\1+", "\1")
    return tf.strings.regex_replace(
        stripped_html, "[%s]" % re.escape(string.punctuation), ""
    )



# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 500

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)


text_ds = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)


AttributeError: ignored

In [ ]:
vocab = np.array(vectorize_layer.get_vocabulary())
vocab[:100]

array(['', '[UNK]', 'يا', 'من', 'على', 'ابن', 'و', 'كل', 'فاشل', 'سلمان',
       'انت', 'والله', 'قلبي', 'قال', 'عاش', 'زمالك', 'يوم', 'واحد',
       'ممحون', 'مش', 'لو', 'لسه', 'كرتونه', 'قلبك', 'فين', 'فيك', 'فدوه',
       'رب', 'حيوان', 'حلو', 'حقيقي', 'حتى', 'بلادي', 'بس', 'انها', 'انا',
       'امي', 'اليمن', 'الله', 'الشرموطه', 'الدوري', 'الحزم', 'احب',
       '😩😭♥️', '😂😂😂', '😁', '🎼', '✋️', '⚪🔴', 'يمن', 'يمكن', 'يغني', 'يعني',
       'يروح', 'يحرق', 'يجيبه', 'يارب', 'ياخد', 'ي', 'ويلا', 'ويكسر',
       'ويع', 'وقدره', 'وعائلتك', 'وش', 'وراعيها❤️', 'وجوهكم', 'وجع',
       'وبنزينكم', 'واقفه👌', 'واستاذن', 'هيعملوا', 'هوا', 'ههههههه',
       'هنااااا', 'هما', 'هلالي', 'هذاكر', 'هاهاهاهاها', 'هالحلقه',
       'نكسب', 'نقف', 'نفضل', 'نفسي', 'نصابين', 'نجم', 'نتعادل', 'ناكها',
       'ميجيش', 'موزه', 'مهم', 'منه', 'منكم', 'منخسرش', 'ممحوون',
       'ممحونه', 'ملك', 'معفنين', 'معرص', 'مصري'], dtype='<U11')

In [ ]:
# # s=tf.strings.regex_replace("مشكووووووووور",r'(\w)\1*','')
# repeat_pattern = re.compile(r'(\w)\1*')
# y=tf.strings.regex_full_match(
#     "happpppppppy",
#     "(\w)\1*",
#     name=None
# )
# # str(s, encoding='cp1252')
# # s.decode('utf8').encode('latin1').decode('utf8')
# # s=tf.strings.unicode_transcode(str(s), "US ASCII", "UTF-8").numpy()
# # print()
# # print(u''.encode('utf-8')+s)
# # y=tf.constant("مشكووووووووووور")
# print(y)


In [ ]:
# # re.sub(r'(.)\1+', r'\1', "haaaaapppppyyy")  
# # re.sub(r'(\w)\1+', r'\1', "مشكووووووووووور")  
# repeat_pattern = re.compile(r'(\w)\1*')
# repeat_pattern.sub(r'\1','مشكووووووور')

In [ ]:
vocab = np.array(vectorize_layer.get_vocabulary())
vocab[:100]

array(['', '[UNK]', 'يا', 'من', 'على', 'ابن', 'و', 'كل', 'فاشل', 'سلمان',
       'انت', 'والله', 'قلبي', 'قال', 'عاش', 'زمالك', 'يوم', 'واحد',
       'ممحون', 'مش', 'لو', 'لسه', 'كرتونه', 'قلبك', 'فين', 'فيك', 'فدوه',
       'رب', 'حيوان', 'حلو', 'حقيقي', 'حتى', 'بلادي', 'بس', 'انها', 'انا',
       'امي', 'اليمن', 'الله', 'الشرموطه', 'الدوري', 'الحزم', 'احب',
       '😩😭♥️', '😂😂😂', '😁', '🎼', '✋️', '⚪🔴', 'يمن', 'يمكن', 'يغني', 'يعني',
       'يروح', 'يحرق', 'يجيبه', 'يارب', 'ياخد', 'ي', 'ويلا', 'ويكسر',
       'ويع', 'وقدره', 'وعائلتك', 'وش', 'وراعيها❤️', 'وجوهكم', 'وجع',
       'وبنزينكم', 'واقفه👌', 'واستاذن', 'هيعملوا', 'هوا', 'ههههههه',
       'هنااااا', 'هما', 'هلالي', 'هذاكر', 'هاهاهاهاها', 'هالحلقه',
       'نكسب', 'نقف', 'نفضل', 'نفسي', 'نصابين', 'نجم', 'نتعادل', 'ناكها',
       'ميجيش', 'موزه', 'مهم', 'منه', 'منكم', 'منخسرش', 'ممحوون',
       'ممحونه', 'ملك', 'معفنين', 'معرص', 'مصري'], dtype='<U11')

In [ ]:
encoded_example = vectorize_layer(text_batch)[:20].numpy()
encoded_example

array([[268,  76,  64, ...,   0,   0,   0],
       [  2, 257, 149, ...,   0,   0,   0],
       [ 26,   2, 214, ...,   0,   0,   0],
       ...,
       [  2,  27,   2, ...,   0,   0,   0],
       [ 24,   2, 181, ...,   0,   0,   0],
       [209, 226,   2, ...,   0,   0,   0]])

In [ ]:
for n in range(5):
  print("Original: ", text_batch[n].numpy().decode())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  RT @USER: جالس أسمع أحمد قاسم يغني: "أحبك من كل قلبي يا بلادي يا يمن". حتى أنا والله أحب اليمن من كل قلبي أحب بلادي من طرفها لطرفها "ب…	NOT_OFF	NOT_HS

Round-trip:  اعلامي هلالي وش ترجي منه اي والله الى قال محمد العويس حراق🔥🔥🔥حارق قلبك يا زقان يا المطي ويع من حرق عساه للحرق يا رب جعلني اشوف فيك يوم اسود انت وعائلتك يا ابن الكلب                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

Original:  يا بكون بحياتك الأهم يا إما ما بدي أكون 🎼	NOT_OFF	NOT_HS

Round-trip:  يا التاج ع الراس يا الساده يا مالك الروح وراعيها❤️                                                            

In [ ]:
# from numpy import array
# from numpy import asarray
# from numpy import zeros

# embeddings_dictionary = dict()
# glove_file = open('E:/Datasets/Word Embeddings/glove.6B.100d.txt', encoding="utf8")

# Create the model

In [ ]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(len(vectorize_layer.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

history = model.fit(raw_train_ds, epochs=30,
                    validation_data=raw_val_ds, 
                    validation_steps=30,
                    callbacks=[EarlyStopping(monitor='val_loss',patience=5)]
                    )


# history=model.fit(train_ds, validation_data=val_ds, epochs=epochs,
#           callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/30
1/1 [==============================] - 18s 18s/step - loss: 0.6920 - accuracy: 0.6190 - val_loss: 0.6957 - val_accuracy: 0.4000
Epoch 2/30
1/1 [==============================] - 0s 67ms/step - loss: 0.6932 - accuracy: 0.6190
Epoch 3/30
1/1 [==============================] - 0s 66ms/step - loss: 0.6915 - accuracy: 0.6190
Epoch 4/30
1/1 [==============================] - 0s 67ms/step - loss: 0.6910 - accuracy: 0.6190
Epoch 5/30
1/1 [==============================] - 0s 68ms/step - loss: 0.6910 - accuracy: 0.6190
Epoch 6/30
1/1 [==============================] - 0s 65ms/step - loss: 0.6890 - accuracy: 0.6190
Epoch 7/30
1/1 [==============================] - 0s 79ms/step - loss: 0.6919 - accuracy: 0.6190
Epoch 8/30
1/1 [==============================] - 0s 68ms/step - loss: 0.6890 - accuracy: 0.6190
Epoch 9/30
1/1 [==============================] - 0s 74ms/step - loss: 0.6904 - accuracy: 0.6190
Epoch 10/30
1/1 [==============================] - 0s 71ms/step - loss: 0.6890 - accu

In [ ]:
test_loss, test_acc = model.evaluate(raw_test_ds)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

#epochs_range = range(22)

plt.figure(figsize=(15, 15))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
sample_text = (' يا حبيبي بكل صفاتہ يا دموع و يا خضوع  ')
predictions = model.predict(np.array([sample_text]))
predictions
#NOT hate speech

In [ ]:
sample_text = (' القم يا ايطالي يا ابن الكلب انت و باصك يلعن اهلك  ')
predictions = model.predict(np.array([sample_text]))
predictions
#Hate